## Python Importation

In [ ]:
import os

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import bk.compute
import bk.plot
import bk.load

import scipy.signal
import neuroseries as nts

## Magic Command

In [ ]:
%matplotlib qt

## Session Init

In [ ]:
path = '/media/billel/GGLab-01-BK/DATA/Rat08-20130713/'
os.chdir(path)
bk.load.path = path
bk.load.rat = 8
bk.load.session = 'Rat08-20130713'
bk.load.day = 6

In [4]:
bk.load.current_session()

Rat : 8 on day : 6
Working with session Rat08-20130713 @ Z:\Rat08\Rat08-20130713


True

## Loading data

In [146]:
neurons, metadata = bk.load.loadSpikeData(bk.load.path)
states = bk.load.states()

Data already saved in Numpy format, loading them from here:
Rat08-20130713-neurons.npy
Rat08-20130713-metadata.npy


## Computing spectrograms

In [6]:
start = 15516.0
end = 16405.0

# start = 16120
# end = 16200
interval = nts.IntervalSet(start,end,time_units='s')

In [7]:
lfp = bk.load.lfp(start,end,166,13)
f,t,Sxx = scipy.signal.spectrogram(lfp.as_units('s'),fs=1250,mode= 'psd',nperseg=10000,noverlap=5000)
t = t +lfp.as_units('s').index[0]
spec = Sxx[f<20]
spec_corr = np.corrcoef(spec,rowvar=False)

Load LFP from Rat08-20130713.lfp


## Computing Neurons correlations

In [8]:
# start = 15516.0
# end = 16405.0

start = 2300
end = 2500

# start = 16120
# end = 16200
interval = nts.IntervalSet(start,end,time_units='s')

# stru = ['Hpc','BLA','Pir']
# channel = [13,101,80]

stru =  ['BLA']
channel = [103]
for n,(s,c) in enumerate(zip(stru,channel),0):
    lfp = bk.load.lfp(start,end,166,c)
    f,t,Sxx = scipy.signal.spectrogram(lfp.as_units('s'),fs=1250,mode= 'psd',nperseg=1000,noverlap=250)
    t = t +lfp.as_units('s').index[0]
    spec = Sxx[f<100]
    spec_corr = np.corrcoef(spec,rowvar=False)
    
    stru_neurons,t_n = bk.compute.binSpikes(neurons[metadata.Region == s],2,start,end)
    neurons_corr = np.corrcoef(stru_neurons,rowvar=False)
    
    plt.figure()
    sub = plt.subplot(4,1,1)
    plt.title('LFP')
    plt.plot(lfp.as_units('s'))
    bk.plot.intervals(states['sws'],col='grey')
    bk.plot.intervals(states['Rem'])
   
    plt.subplot(4,1,2,sharex=sub)
    plt.title('Spectrogram')
    plt.pcolormesh(t, f, Sxx)
    plt.ylim(0,100)
    plt.clim(0,5000)
    
    
    plt.subplot(4,1,3,sharex=sub)
    plt.title('SpectrogramCorr')
    plt.pcolormesh(t,t,spec_corr)
    plt.clim(0,1)
    
    plt.subplot(4,1,4,sharex = sub)
    plt.title('SpikeTrainCorr') 
    plt.pcolormesh(t_n,t_n,neurons_corr)
    plt.xlim(start,end)
    
    plt.tight_layout()
    plt.suptitle(s)

Load LFP from Rat08-20130713.lfp


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.


In [32]:
stru_neurons.shape

(94, 999)

In [10]:
fig, axes = plt.subplots(nrows=3, ncols=2)

In [16]:
for ax in axes.flatten():
    ax.plot(lfp)

In [31]:
states['sws'].end

0      1697000000
1      1926000000
2      2429000000
3      3275000000
4      3841000000
5      3916000000
6      3992000000
7      4529000000
8      4648000000
9      5426000000
10     6069000000
11     6629000000
12     7311000000
13     7759000000
14     7909000000
15     8797000000
16     9333000000
17    16145000000
18    16765000000
19    17259000000
20    17429000000
21    17669000000
22    17764000000
23    17810000000
24    18076000000
25    18528000000
26    18979000000
27    19167000000
28    19448000000
29    19771000000
30    20363000000
31    21286000000
32    21629000000
33    21850000000
Name: end, dtype: int64

In [9]:
end  = nts.Ts(np.array(states['sws'].end+1_000_001))
a = states['Rem'].in_interval(end)

In [10]:
states['sws'].iloc[1]

start    1777000000
end      1926000000
Name: 1, dtype: int64

In [11]:
states['Rem'].iloc[0]

start    1927000000
end      2013000000
Name: 0, dtype: int64

In [12]:
interval.

SyntaxError: invalid syntax (<ipython-input-12-5ceda6e46fc5>, line 1)

In [14]:
import itertools

In [15]:
for j in itertools.permutations(states.keys(),2):

SyntaxError: unexpected EOF while parsing (<ipython-input-15-fdc0e234b070>, line 1)

In [229]:
def state_transitions(states,epsilon = 1):
    '''
        states : dict of nts.Interval_Set
        
        This function compute transition in between Intervals in a dict.
        It returns a new dict with intervals and when the transition occurs
        
        epsilon : tolerance time delay between state
        
    '''
    
    import itertools
    
    for state in states:
        print(state)
        states[state] = states[state].drop_short_intervals(1)
    
    transitions_intervals = {}
    transitions_timing = {}
    
    for items in itertools.permutations(states.keys(),2):
        states[items[0]] = states[items[0]].drop_short_intervals(1)
        states[items[1]] = states[items[1]].drop_short_intervals(1)
        
        print('Looking at transition from',items[0],' to ',items[1])
        end = nts.Ts(np.array(states[items[0]].end + (epsilon * 1_000_000)+1))
        in_next_epoch = states[items[1]].in_interval(end)
        
        transitions_intervals.update({items:[]})
        transitions_timing.update({items:[]})

        for n,t in enumerate(in_next_epoch):
            if np.isnan(t): continue            
            start = states[items[0]].iloc[n].start
            trans = int(np.mean([states[items[0]].iloc[n].end,states[items[0]].iloc[n].start]))
            end  = states[items[1]].iloc[int(t)].end
            transitions_intervals[items].append([start,end])
            transitions_timing[items].append(trans)
        
        if  not transitions_timing[items] == []:      
            transitions_intervals[items] = np.array(transitions_intervals[items])
            transitions_intervals[items] = nts.IntervalSet(transitions_intervals[items][:,0],transitions_intervals[items][:,1])
            
            transitions_timing[items] = nts.Ts(t = np.array(transitions_timing[items]))
    return transitions_intervals,transitions_timing
    

In [230]:
c,d = state_transitions(a)

('Rem', 'sws')
('Rem', 'wake')
('Rem', 'drowsy')
('sws', 'Rem')
('sws', 'wake')
('sws', 'drowsy')
('wake', 'Rem')


AttributeError: 'list' object has no attribute 'drop_short_intervals'

In [228]:
a,b = state_transitions(states)

Rem
sws
wake
drowsy
Looking at transition from Rem  to  sws
Looking at transition from Rem  to  wake
Looking at transition from Rem  to  drowsy
Looking at transition from sws  to  Rem
Looking at transition from sws  to  wake
Looking at transition from sws  to  drowsy
Looking at transition from wake  to  Rem
Looking at transition from wake  to  sws
Looking at transition from wake  to  drowsy
Looking at transition from drowsy  to  Rem
Looking at transition from drowsy  to  sws
Looking at transition from drowsy  to  wake


In [213]:
bk.plot.intervals(a['sws','Rem'])

In [171]:
a = states['drowsy'].drop_duplicates()

In [205]:
c = b['sws','Rem'].as_units('s').index

In [211]:
a['sws','Rem']
b['sws','Rem']

Time (us)
1851500000    NaN
2266500000    NaN
3050500000    NaN
3612000000    NaN
3902000000    NaN
3981000000    NaN
4276500000    NaN
5235000000    NaN
5779000000    NaN
7140500000    NaN
7567000000    NaN
7848500000    NaN
8641000000    NaN
9110000000    NaN
15830500000   NaN
16600500000   NaN
17030000000   NaN
17396500000   NaN
17800500000   NaN
17964000000   NaN
18330000000   NaN
18821000000   NaN
19126500000   NaN
19317500000   NaN
19658500000   NaN
21076500000   NaN
21494000000   NaN
21762500000   NaN
dtype: float64

In [208]:
states

{'Rem':           start          end
 0    1927000000   2013000000
 1    2430000000   2603000000
 2    3276000000   3371000000
 3    3842000000   3887000000
 4    3917000000   3969000000
 5    3993000000   4023000000
 6    4530000000   4567000000
 7    4677000000   4838000000
 8    4864000000   4920000000
 9    5427000000   5488000000
 10   6070000000   6206000000
 11   7312000000   7374000000
 12   7760000000   7787000000
 13   7910000000   8030000000
 14   8798000000   8852000000
 15   9334000000   9402000000
 16  16146000000  16405000000
 17  16766000000  16787000000
 18  17260000000  17363000000
 19  17430000000  17496000000
 20  17811000000  17851000000
 21  18077000000  18131000000
 22  18529000000  18614000000
 23  18980000000  19084000000
 24  19168000000  19186000000
 25  19449000000  19545000000
 26  19772000000  19817000000
 27  21287000000  21358000000
 28  21630000000  21674000000
 29  21851000000  21953000000,
 'sws':           start          end
 0    1448000000   169700

In [212]:
for j in c:
    print(j)
    plt.axvline(j,c='r')

1851.5
2266.5
3050.5
3612.0
3902.0
3981.0
4276.5
5235.0
5779.0
7140.5
7567.0
7848.5
8641.0
9110.0
15830.5
16600.5
17030.0
17396.5
17800.5
17964.0
18330.0
18821.0
19126.5
19317.5
19658.5
21076.5
21494.0
21762.5


In [126]:
a.drop_duplicates()

,start,end
0,546000000,739000000
1,858000000,921000000
2,1056000000,1205000000
3,1698000000,1776000000
4,2014000000,2103000000
5,3372000000,3382000000
6,4649000000,4671000000
7,4839000000,4863000000
8,4921000000,5043000000
9,6945000000,6969000000
